<a href="https://colab.research.google.com/github/afterworldsimulacra/SVGD-QSE/blob/main/PyMC_tomography_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymc

In [ ]:
import pymc as pm
import numpy as np

import plotly.graph_objects as go

In [ ]:
sampled_xyz = np.random.uniform(low=-1, high=1, size=3)
while(np.sum(sampled_xyz**2)>1):
    sampled_xyz = np.random.uniform(low=-1, high=1, size=3)
  # Sample one set of biases

theta = (1 + sampled_xyz) / 2

coin_choice_fixed = np.random.choice([0, 1, 2], size=100, p=np.ones(3) / 3)

data = np.random.binomial(n=1, p=theta[coin_choice_fixed])  # n=1 for Bernoulli trials

In [ ]:
sampled_xyz

array([-0.39486299, -0.50383744, -0.16543999])

In [ ]:
coin_choice_fixed

array([2, 1, 0, 1, 2, 0, 1, 2, 2, 2, 0, 1, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2,
       2, 0, 1, 2, 2, 0, 1, 0, 0, 1, 1, 0, 2, 0, 2, 0, 0, 2, 1, 0, 1, 2,
       0, 0, 2, 0, 2, 1, 2, 1, 1, 0, 0, 1, 2, 1, 2, 0, 1, 2, 1, 1, 1, 2,
       1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 0, 2, 2, 0, 1, 2, 1, 0, 0, 2,
       1, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
data

array([0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0])

In [ ]:
with pm.Model() as model:
    # Prior (Uniform on cube)

    xyz = pm.Uniform('xyz', lower=-1, upper=1, shape=3)
    # are px py pz not beta distributions?
    # Likelihood
    const = pm.Potential("constraint",pm.math.switch(pm.math.le(pm.math.sum(xyz**2),1),0,-np.inf))
    theta = (1 + xyz) / 2



    likelihood = pm.Bernoulli('likelihood', p=theta[coin_choice_fixed], observed=data)


    # Inference
    approx = pm.fit(method=pm.SVGD(n_particles=160, jitter=1.))
    trace = approx.sample(10000)

Output()

In [ ]:
# Extract 'xyz' data using arviz
posterior = trace.posterior
xyz_data = posterior.data_vars['xyz']


# Generate prior samples for plotting (10000 samples for better visualization)
prior_samples = np.random.uniform(low=-1, high=1, size=(10000, 3))

# Visualization (3D Scatterplot with Plotly)
fig = go.Figure(data=[
    go.Scatter3d(  # Posterior
        x=xyz_data[0, :, 0],
        y=xyz_data[0, :, 1],
        z=xyz_data[0, :, 2],
        mode='markers',
        marker=dict(size=2, color='blue', opacity=0.25),
        name='Posterior'
    ),
    go.Scatter3d(  # Prior
        x=prior_samples[:, 0],
        y=prior_samples[:, 1],
        z=prior_samples[:, 2],
        mode='markers',
        marker=dict(size=2, color='orange', opacity=0.25),
        name='Prior'
    ),
    go.Scatter3d(  # True state
        x=[sampled_xyz[0]],
        y=[sampled_xyz[1]],
        z=[sampled_xyz[2]],
        mode='markers',
        marker=dict(size=15, color='red', symbol='cross'),  # Star symbol for the true state
        name='True State'
    )
])

fig.update_layout(
    title='Prior, Posterior, and True State',
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z',
    )
)
fig.show()

In [ ]:
np.mean(posterior)

<xarray.Dataset> Size: 8B
Dimensions:  ()
Data variables:
    xyz      float64 8B -0.1162

In [ ]:
mul(sampled_xyz)

NameError: name 'mul' is not defined